## Imports

In [77]:
import json
import math
from collections import defaultdict, Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi

## Προεπεξεργασία
Εδώ ειναι η συνάρτηση της προεπεξεργασίας του query (ερωτήματος).

In [78]:
# Preprocessing setup
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def preprocess_query(query):
    """Preprocess the query for search."""
    # Tokenize and remove punctuation, digits, and lowercase everything
    tokens = word_tokenize(query.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha()]
    
    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]

    return tokens


## Boolean Retrieval
Παρακάτω είναι η υλοποίηση του αλγορίθμου αναζήτησης Boolean Retrieval. Σκοπός της συνάρτησης ειναι να λάβει μια λίστα από λέξεις-κλειδιά απο ένα ερώτημα αναζήτησης και να εφαρμόσει τους λογικούς τελεστές και να επιστρέψει το σύνολο των εγγράφων που ταιριάζουν με το ερώτημα. Οι παράμετροι έιναι οι:
query_tokens: Μια λίστα από λέξεις (tokens) του ερωτήματος.
inverted_index: Ένα λεξικό που αντιστοιχεί σε λέξεις (όρους) σε σύνολα εγγράφων στα οποία εμφανίζονται.

Ορίζει τελεστές, αρχικοποιεί στοίβα, επεξεργάζεται κάθε token, ελέγχει για τελεστές, και επεξεργάζεται λέξεις.


In [79]:
def boolean_retrieval(query_tokens, inverted_index):
    """Boolean retrieval (supports AND, OR, NOT)."""
    operators = {"AND", "OR", "NOT"}
    stack = []

    # Για να αποθηκεύσουμε τους τίτλους που βρέθηκαν
    titles_found = set()

    for token in query_tokens:

        if token.upper() in operators:

            # Handle AND operation
            if token.upper() == "AND":
                set2 = stack.pop() if stack else set()
                set1 = stack.pop() if stack else set()
                stack.append(set1 & set2)

            # Handle OR operation
            elif token.upper() == "OR":
                set2 = stack.pop() if stack else set()
                set1 = stack.pop() if stack else set()
                stack.append(set1 | set2)

            # Handle NOT operation
            elif token.upper() == "NOT":
                set1 = stack.pop() if stack else set()
                all_docs = set(inverted_index.keys())
                stack.append(all_docs - set1)

        else:
            # Αν η λέξη δεν είναι operator, τότε είναι λέξη για αναζήτηση στο inverted index
            term_docs = inverted_index.get(token, set())

            # Ενημερώνουμε τη λίστα των τίτλων που βρέθηκαν
            titles_found.update(term_docs)
            stack.append(term_docs)ex

    # Επιστρέφουμε τη λίστα τίτλων που βρέθηκαν
    result_titles = list(titles_found)
    return result_titles



## TFIDF
Η συνάρτηση compute_tfidf_ranking() χρησιμοποιεί την τεχνική TF-IDF (Term Frequency - Inverse Document Frequency) για να κατατάξει τα έγγραφα βάσει της σημασίας τους σε σχέση με το ερώτημα. 
Παράμετροι: 
query_tokens: Μια λίστα με τις λέξεις (tokens) του ερωτήματος που θέλουμε να αναζητήσουμε.
documents: Μια λίστα από έγγραφα, όπου κάθε έγγραφο είναι ένα λεξικό που περιέχει τουλάχιστον το τίτλο του ('title') και το επεξεργασμένο περιεχόμενό του ('processed_content').
Αρχικοποίηση του TfidfVectorizer ο οποίος είναι ένα εργαλείο από τη βιβλιοθήκη scikit-learn που μετατρέπει τα έγγραφα σε διανύσματα TF-IDF.
Εξαγωγή τίτλων και περιεχομένων από τα έγγραφα, υπολογισμός του πίνακα TF-IDF ο οποίος περιέχει τα TF-IDF διανύσματα όλων των εγγράφων.
Μετατροπή του ερωτήματος σε διανύσμα TF-IDF, υπολογισμός των σκορ, ταξινόμηση και επιστροφή των εγγράφων.

In [80]:



def compute_tfidf_ranking(query_tokens, documents):
    """Rank documents using TF-IDF."""
    vectorizer = TfidfVectorizer()
    doc_titles = [doc['title'] for doc in documents]
    doc_contents = [doc['processed_content'] for doc in documents]

    tfidf_matrix = vectorizer.fit_transform(doc_contents)
    query_vector = vectorizer.transform([" ".join(query_tokens)])

    scores = (tfidf_matrix @ query_vector.T).toarray().flatten()

    ranked_docs = sorted(
        [(doc_titles[i], scores[i]) for i in range(len(doc_titles))],
        key=lambda x: x[1],
        reverse=True
    )
    return ranked_docs


## BM25OKAPI
Η συνάρτηση compute_bm25_ranking() χρησιμοποιεί τον αλγόριθμο BM25 για να κατατάξει τα έγγραφα.
Παράμετροι ίδιοι με τον TF-IDF. 
Η υλοποίηση αποτελείται από:
Εξαγωγή των περιεχομένων των εγγράφων, δημιουργία του μοντέλου BM25, υπολογισμός των σκορ BM25, ταξινόμηση των εγγράφων με βάση τα σκορ, επιστροφή των καταταγμένων εγγράφων

In [81]:

def compute_bm25_ranking(query_tokens, documents):
    """Rank documents using BM25."""
    doc_contents = [doc['processed_content'].split() for doc in documents]
    bm25 = BM25Okapi(doc_contents)
    scores = bm25.get_scores(query_tokens)

    ranked_docs = sorted(
        [(documents[i]['title'], scores[i]) for i in range(len(documents))],
        key=lambda x: x[1],
        reverse=True
    )
    return ranked_docs


## Vector Space Model (VSM)
Η συνάρτηση compute_vsm_ranking() χρησιμοποιεί το Vector Space Model (VSM) για να κατατάξει τα έγγραφα με βάση την ομοιότητα τους με το ερώτημα, χρησιμοποιώντας τον cosine similarity. Οι παράμετροι παραμένουν ίδιοι, όμως με την πρόσθεση του inverted_index ο οποίος είναι ένας αντίστροφος δείκτης που καταγράφει σε ποιά έγγραφα εμφανίζονται οι λέξεις του εγγράφου.
Η υλοποίηση αποτελείται από:
Υπολογισμός Term Frequency (TF), Υπολογισμός Inverse Document Frequency (IDF), Υπολογισμός των βαρών TF-IDF για κάθε έγγραφο, Υπολογισμός των βαρών TF-IDF για το ερώτημα, υπολογισμός cosine similarity, ταξινόμηση των εγγράφων

In [82]:

def compute_vsm_ranking(query_tokens, documents, inverted_index):
    """Rank documents using the Vector Space Model (cosine similarity)."""
    # Step 1: Calculate term frequency (TF)
    tf = defaultdict(Counter)
    doc_lengths = defaultdict(int)

    for doc_id, doc in enumerate(documents):
        tokens = doc['processed_content'].split()
        for word in tokens:
            tf[doc_id][word] += 1
            doc_lengths[doc_id] += 1

    # Step 2: Calculate inverse document frequency (IDF)
    idf = {}
    total_docs = len(documents)
    for word in inverted_index:
        idf[word] = math.log(total_docs / (1 + len(inverted_index[word])))  # Added smoothing to avoid division by zero

    # Step 3: Calculate TF-IDF weights for documents
    tfidf_weights = defaultdict(dict)
    for doc_id in tf:
        for word, freq in tf[doc_id].items():
            tfidf_weights[doc_id][word] = (freq / doc_lengths[doc_id]) * idf.get(word, 0)

    # Step 4: Calculate TF-IDF weights for the query
    query_tf = Counter(query_tokens)
    query_length = sum(query_tf.values())
    query_weights = {word: (query_tf[word] / query_length) * idf.get(word, 0) for word in query_tf}

    # Step 5: Compute cosine similarity
    scores = defaultdict(float)
    for doc_id, weights in tfidf_weights.items():
        dot_product = sum(weights[word] * query_weights.get(word, 0) for word in query_weights if word in weights)
        doc_norm = math.sqrt(sum(value ** 2 for value in weights.values()))
        query_norm = math.sqrt(sum(value ** 2 for value in query_weights.values()))
        if doc_norm * query_norm != 0:  # Avoid division by zero
            scores[doc_id] = dot_product / (doc_norm * query_norm)

    # Rank documents by score
    ranked_docs = sorted(
        [(documents[doc_id]['title'], score) for doc_id, score in scores.items()],
        key=lambda x: x[1],
        reverse=True
    )
    return ranked_docs


Εδώ απλά φορτώνονται τα .json αρχεία που χρειαζόμαστε.


In [83]:

# Load the inverted index and preprocessed articles
with open("inverted_index.json", "r", encoding="utf-8") as f:
    inverted_index = json.load(f)

with open("preprocessed_wikipedia_articles.json", "r", encoding="utf-8") as f:
    documents = json.load(f)


Παρακάτω είναι η υλοποίηση του συντονιστή της μηχανής αναζήτησης.

In [84]:

def search_engine(query, method="boolean"):
    """Search engine supporting Boolean, TF-IDF, BM25, and VSM."""
    query_tokens = preprocess_query(query)

    if method == "boolean":
        result_titles = boolean_retrieval(query_tokens, inverted_index)
        results = [doc for doc in documents if doc['title'].lower() in [title.lower() for title in result_titles]]
        return [doc for doc in documents if doc['title'] in result_titles]
       
    elif method == "tfidf":
        ranked_docs = compute_tfidf_ranking(query_tokens, documents)
        return ranked_docs

    elif method == "bm25":
        ranked_docs = compute_bm25_ranking(query_tokens, documents)
        return ranked_docs

    elif method == "vsm":
        ranked_docs = compute_vsm_ranking(query_tokens, documents, inverted_index)
        return ranked_docs

    else:
        raise ValueError("Invalid method. Choose from 'boolean', 'tfidf', 'bm25', or 'vsm'.")


Τέλος, είναι το GUI.

In [85]:

if __name__ == "__main__":
    print("Welcome to the search engine!\n")

    while True:
        user_query = input("Enter your query (or 'exit' to exit): ")
        if user_query.lower() == "exit":
            print("Exiting the search engine.")
            break

        method = input("Choose search method (boolean, tfidf, bm25): ").lower()
        
        try:
            # Αμέσως επεξεργαζόμαστε την αναζήτηση και παίρνουμε τα αποτελέσματα
            results = search_engine(user_query, method)
            
            # Εκτύπωση των αποτελεσμάτων ως λίστα
            print(f"\nResults ({len(results)} found):")
            titles = []
            if method == "boolean":
                # Όταν είναι boolean, παίρνουμε τους τίτλους από τα αποτελέσματα
                for result in results:
                    titles.append(result['title'])
            else:
                # Για άλλες μεθόδους, παίρνουμε τίτλους και σκορ
                for title, score in results:
                    titles.append(f"{title} (score: {score:.4f})")

            # Εκτύπωση της λίστας με τους τίτλους
            print("\n".join([f"- {title}" for title in titles]))

        except Exception as e:
            print(f"Error: {e}")


Welcome to the search engine!


Results (1000 found):
- Computer (score: 0.4856)
- Computer science (score: 0.4578)
- Computing (score: 0.3993)
- Computer hardware (score: 0.3736)
- Computer architecture (score: 0.3302)
- Computer vision (score: 0.2749)
- Computer graphics (score: 0.2463)
- Theoretical computer science (score: 0.2333)
- Computer simulation (score: 0.2230)
- Microcomputer (score: 0.2226)
- Security hacker (score: 0.1901)
- Joseph Weizenbaum (score: 0.1759)
- Computational mathematics (score: 0.1696)
- Wetware computer (score: 0.1678)
- Parallel computing (score: 0.1670)
- Linda Shapiro (score: 0.1665)
- Human–computer interaction (score: 0.1655)
- Human–computer interaction (score: 0.1655)
- Brian Cantwell Smith (score: 0.1655)
- Computer accessibility (score: 0.1486)
- Distributed computing (score: 0.1429)
- IBM 701 (score: 0.1404)
- Machine perception (score: 0.1393)
- Arthur Samuel (computer scientist) (score: 0.1383)
- Turing completeness (score: 0.1366)
- Pat Hayes